## DS102 Project

#### Research Topic
Investigating how temperature, humidity and rainfall affects tourist arrivals to Singapore. 

#### Reason for choice of topic

#### Dataset 
1. Monthly Visitor Arrivals from Singapore Tourism Board in the form of Excel spreadsheet
2. Realtime Weather Readings from Jan 2017 to Nov 2018 across Singapore from data.gov.sg in the form of APIs
    - Air Temperature
    - Relative Humidity 
    - Rainfall  
    
#### Challenges 

1. Data from data.gov was incomplete (2018-02-07) had no data 
2. Getting temp banned by data.gov

error at 2018-02-07
error at 2018-02-18
error at 2018-04-12
error at 2018-04-13
error at 2018-04-14
error at 2018-04-15

In [145]:
# import libraries 
import pandas as pd 
import numpy as np 
import matplotlib
import datetime 
import requests 
import csv
import time


In [150]:
# making API calls

def get(url): 
    resp = requests.get(url)
    if resp.ok:
        return resp.json()
    

date = datetime.date(2016,12,31)
date_list = []
url_list = [] # storing all the urls 
for i in range(730): # 730 is from jan 2017 to dec 2018 
    date += datetime.timedelta(days=1) # looping to increase days by 1
    start_url = f"https://api.data.gov.sg/v1/environment/air-temperature?date={date}"
    url_list.append(start_url) 
    date_list.append(date)


In [149]:
value_list = [] # contains all values in list of list 
mean_list = [] # contains all mean temperature by date 
data_dict = {} # dict containing date and mean temperature
debug_url = []
for url in (url_list): # narrowing down to get value for that time of the day 
    data = get(url) 
    time.sleep(5)
    debug_url.append(url)
    try:
        datestamp = (data["items"][0]["timestamp"])[0:10] 
        for info in data["items"]:  
            #narrowing data down to working hours of stocks exchange
            if info.get("timestamp")[11:19] >="09:00:00" and info.get("timestamp")[11:19] < "17:00:00": 
                readings = [info.get("readings")] 
                for list1 in readings:
                    for dicts in list1: 
                        value_list.append(dicts.get("value"))
                mean_temp = round((sum(value_list)/len(value_list)), 2)
                data_dict[datestamp] = [mean_temp]
                time.sleep(40)
    except: 
        IndexError # ignoring empty datasets

df = pd.DataFrame.from_dict(data_dict, orient="index", columns = ["Mean Temperature"])
df.to_csv("Mean_Temperature_During_Day.csv")

df




,Mean Temperature
2017-01-01,28.87
2017-01-02,28.25
2017-01-03,28.25
2017-01-04,28.32
2017-01-05,28.61
